## Notebook responsável pela geração do mapa utilizado pelo SVAA

In [ ]:
import os

import folium
import numpy as np
import pandas as pd
import requests as req
from branca.element import MacroElement
from folium.plugins import Fullscreen
from jinja2 import Template
from resolve_path import ajuste_path, read_input

### Carrega Datasets

In [ ]:
pathUtil = ajuste_path('data/util/')

df_coord = pd.read_csv(pathUtil + 'local_coordenada.csv',
                       sep=';', encoding='utf-8')

df_mapa = pd.read_csv(pathUtil + 'dataset_mapa.csv', sep=',')

# Remover as linhas onde as colunas 'latitude' ou 'longitude' estejam NaN
# df_mapa = df_mapa.dropna(subset=['latitude', 'longitude'])

## Preparação do Mapa

### Preparação do df_lt

In [ ]:
# Seleciona apenas as linhas de transmissão do df_coord
df_lt = df_coord[df_coord['local_de_instalacao'].apply(
    lambda x: x.startswith('S-L'))].copy()

# Filtra o df_lt para conter apenas Linhas de Transmissão que estão contidas no df_mapa
def filtra_linhas_transmissao(df_lt, df_mapa):
    # Cria uma lista com os locais de instalação do df_mapa
    lista_locais = df_mapa['local_de_instalacao'].unique()
    # Filtra o df_lt para conter apenas Linhas de Transmissão que estão contidas no df_mapa
    lts_contida_em_ambos_datasets = df_lt['lt'].isin(lista_locais)

    df_lt_filtrado = df_lt[lts_contida_em_ambos_datasets]

    return df_lt_filtrado

# Função auxiliar para extrair a LT de um vão
def extrai_prefixo_lt(valor):
    partes = valor.split('-')
    lt = '-'.join(partes[:5])
    return lt

# Adiciona uma coluna com o prefixo de cada linha de transmissão (ex: 'S-L-ARE/CBA-C1-LT525-V0372' -> 'S-L-ARE/CBA-C1-LT525')
df_lt['lt'] = df_lt['local_de_instalacao'].apply(extrai_prefixo_lt)

df_lt_filtrado = filtra_linhas_transmissao(df_lt, df_mapa)


print("Número de linhas após a filtragem: " + str(df_lt_filtrado.shape[0]))

### Definição das cores que serão utilizadas no mapa 

In [ ]:
corMaisFraca = "#dfc27d"
corMediaFraca = "#bf812d"
corMediaForte = "#8c510a"
corMaisForte = "#543105"


cores_intermediarias_gradiente = [
    corMaisFraca, corMediaFraca, corMediaForte, corMaisForte]


def get_color_discrete(score):
    if score is np.nan:
        return "#000000"
    elif score < 0.25:
        return corMaisFraca
    elif score < 0.5:
        return corMediaFraca
    elif score < 0.75:
        return corMediaForte
    else:
        return corMaisForte

### Funções utilizadas pelo mapa

In [ ]:
# Função que adiciona camadas ao mapa (camadas atuais: poligonos do brasil, substações) TO DO: adicionar acidentes e linhas de transmissão
def adiciona_camadas(map_object, df, nome_camada):
    '''
    Função que adiciona camadas ao mapa.

    Parâmetros:
    - map_object: objeto folium.Map
        O objeto do mapa ao qual as camadas serão adicionadas.
    - df: pandas.DataFrame
        DataFrame contendo os dados que serão usados para adicionar as camadas.
        Deve conter as colunas 'latitude', 'longitude', 'local_de_instalacao', 'mes', 'ano', 'hh_total' e 'probabilidade'.
    - nome_camada: str
        O nome da camada que será adicionada ao mapa.

    Adiciona um marcador ao mapa para cada local de instalação no DataFrame data.
    Se for uma linha de transmissão, o marcador será um losangulo, se não será um circulo.
    '''
    camada = folium.FeatureGroup(name=nome_camada)
    for _, row in df.iterrows():
        color = get_color_discrete(row["probabilidade"])

        # Altera a cor da fonte do popup de acordo com a cor do marcador
        font_color = '#543105'
        if color in ["#543105", "#8c510a"]:
            font_color = '#dfc27d'
        else:
            if color in ["#dfc27d"]:
                font_color = '#8c510a'
        popup_content = f"""
            <div>
                <div style="font-size: 1.1em;">
                    <b>Local:</b> {row['local_de_instalacao']}<br>
                </div>
                <div style="margin-top: 10px;">
                    <b>HH total por mês em {(row['mes'])}/{(row['ano'])}:</b> {round(row['hh_total'], 2)}<br>
                    <b>Probabilidade:</b> {round(row['probabilidade'], 2)}
                </div>
            </div>
        """
        if row['local_de_instalacao'].startswith('S-L'):  # Se for uma linha de transmissão
            html = f"""
            <div class="map-marker vao" data-local="{row['local_de_instalacao']}" style="
                    width: 0;
                    height: 0;
                    border-left: 12px solid transparent;
                    border-right: 12px solid transparent;
                    position: relative;
                    cursor: pointer;
                    transition: transform 0.3s ease;
                " onmouseover="this.style.transform='scale(1.2)'"
                onmouseout="this.style.transform='scale(1)'">
                    <div style="
                        background-color: {color};
                        color: {font_color};
                        border-radius: 50%;
                        width: 40px;
                        height: 40px;
                        display: flex;
                        justify-content: center;
                        align-items: center;
                        font-size: 10px;
                        font-weight: bold;
                        transform: translate(-50%, -50%);
                        clip-path: polygon(50% 0%, 100% 50%, 50% 100%, 0% 50%);
                    ">
                        {round(row['probabilidade'] * 100, 2)}%
                    </div>
            """
        else:  # Se for uma subestação, desenha um círculo
            html = f"""
                <div class="map-marker substacao" data-local="{row['local_de_instalacao']}" style="
                width: 0;
                height: 0;
                border-left: 12px solid transparent;
                border-right: 12px solid transparent;
                position: relative;
                cursor: pointer;
                transition: transform 0.3s ease;
            " onmouseover="this.style.transform='scale(1.2)'"
            onmouseout="this.style.transform='scale(1)'">
                    <div style="
                        background-color: {color};
                        color: {font_color};
                        border-radius: 50%;
                        width: 40px;
                        height: 40px;
                        display: flex;
                        justify-content: center;
                        align-items: center;
                        font-size: 10px;
                        font-weight: bold;
                        transform: translate(-50%, -50%);
                    ">
                        {round(round(row['probabilidade'], 2)*100, 2)}%
                    </div>
            """

        # Adiciona o conteúdo ao mapa
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            rise_on_hover=True,
            icon=folium.DivIcon(html=html),
            popup=folium.Popup(popup_content, max_width=2650),
            zIndexOffset=round(row["probabilidade"] * 100),
            tooltip=row['local_de_instalacao']
        ).add_to(camada)

        map_object.add_child(camada)


# Função que define o período de tempo para a visualização do mapa


def filtra_df_por_tempo(df, mes, ano):
    df = df.query("`mes` == @mes and `ano` == @ano")
    return df


def adiciona_vaos_lt(map_object, df_linhas_de_transmissao, cor):
    '''
    Função que adiciona os vãos de cada linha de transmissão ao mapa.
    '''
    lista_coordenadas = []
    # BACALHAU TO DO: fazer funcionar retirar os vaos de linhas de transmissão
    camada = folium.FeatureGroup(name='Vãos das Linhas de Transmissão')
    for group, lt in df_linhas_de_transmissao.groupby('lt'):
        lt = lt.reset_index(drop=True)
        for _, row in lt.iterrows():
            lista_coordenadas.append((row['latitude'], row['longitude']))

        folium.PolyLine(
            lista_coordenadas,
            color=cor,
            weight=3,
            opacity=0.8,
            tooltip=group,
            no_clip=True,
        ).add_to(camada)

        lista_coordenadas = []

    map_object.add_child(camada)

In [ ]:
# Função para adicionar um marcador com o acidente ao mapa
def adiciona_acidente(mapa, row, color):
    folium.Marker(
        location=[row['latitude_acidente'], row['longitude_acidente']],
        popup=f"<b>Local Associado:</b> {row['local_de_instalacao']}<br><b>Potencial: {row['potencial_acidente']}<br><b>Mes do Acidente:</b> {row['data_acidente']}",
        rise_on_hover=True,
        zIndexOffset=1,
        tooltip=row['local_de_instalacao'],
        icon=folium.DivIcon(
            html=f"""
            <div class="map-marker acidente" data-local="{row['local_de_instalacao']}" style="
                width: 0;
                height: 0;
                border-left: 12px solid transparent;
                border-right: 12px solid transparent;
                border-bottom: 24px solid {color};
                position: relative;
                cursor: pointer;
                transition: transform 0.3s ease;
            " onmouseover="this.style.transform='scale(1.2)'"
            onmouseout="this.style.transform='scale(1)'">
                <div style="
                    position: absolute;
                    top: 5px;
                    left: 50%;
                    transform: translate(-50%, 0);
                    color: white;
                    font-size: 14px;
                    font-weight: bold;
                ">
                    !
                </div>
            </div>
            """
        )
    ).add_to(mapa)

# Adiciona o CSS e JavaScript ao marcador quando ele for clicado
# TO DO: ver de colocar o triangulo para piscar ao inves do highlight quando clicado?
def destaca_marcador_selecionado(mapa):
    highlight_script = """
<style>
    /* Triângulo para destacar Acidente */
    .map-marker.acidente.highlight::after {
        content: '';
        position: absolute;
        top: -8px;
        left: -20px;
        width: 0;
        height: 0;
        border-left: 20px solid transparent;
        border-right: 20px solid transparent;
        border-bottom: 36px solid rgba(0, 0, 255, 0.3);
        z-index: -1;
        animation: pulse 1s infinite;
    }

/* Losango para destacar Vão */
.map-marker.vao.highlight::after {
    content: '';
    position: absolute;
    top: -25px; /* Ajuste a posição conforme necessário */
    left: -25px; /* Ajuste a posição conforme necessário */
    width: 50px; /* Largura do losango */
    height: 50px; /* Altura do losango */
    background: rgba(0, 0, 255, 0.3); /* Cor do losango */
    clip-path: polygon(50% 0%, 100% 50%, 50% 100%, 0% 50%); /* Forma de losango */
    z-index: 10000;
    animation: pulse 1s infinite;
    display: block;
}

    @keyframes pulse {
        0% {
            transform: scale(1);
            opacity: 1;
        }
        50% {
            transform: scale(1.2);
            opacity: 0.7;
        }
        100% {
            transform: scale(1);
            opacity: 1;
        }
    }

    /* Circulo para destacar Subestação */
    .highlight-circle {
        content: '';
        position: absolute;
        top: -25px;
        left: -25px;
        width: 50px;
        height: 50px;
        border-radius: 50%;
        background: rgba(0, 0, 255, 0.3);
        transform: translate(-50%, -50%);
        z-index: 10000;
        animation: pulse 1s infinite;
        display: block;
    }
</style>
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            const markers = document.querySelectorAll('.map-marker');
            markers.forEach(marker => {
                marker.addEventListener('click', function() {
                    // Remove destaque de todos os marcadores
                    markers.forEach(m => {
                        m.classList.remove('highlight');
                        const highlightCircle = m.querySelector('.highlight-circle');
                        if (highlightCircle) {
                            highlightCircle.style.display = 'none'; // Esconde o círculo
                        }
                        const highlightDiamond = m.querySelector('.highlight-diamond');
                        if (highlightDiamond) {
                            highlightDiamond.style.display = 'none'; // Esconde o losango
                        }
                    });

                    // Aplica destaque ao marcador clicado
                    this.classList.add('highlight');

                    // Se for um marcador de subestação, adiciona um círculo de destaque
                    if (this.classList.contains('substacao')) { // Verifica se é uma subestação
                        let highlightCircle = this.querySelector('.highlight-circle');
                        if (!highlightCircle) {
                            highlightCircle = document.createElement('div');
                            highlightCircle.className = 'highlight-circle';
                            this.appendChild(highlightCircle); // Adiciona o círculo ao marcador
                        }
                        highlightCircle.style.display = 'block'; // Mostra o círculo
                        // Remove destaque dos triângulos (se houver)
                        this.classList.remove('highlight');

                    // Destaca os acidentes correspondentes
                    const localDeInstalacao = this.getAttribute('data-local');
                    markers.forEach(m => {
                        if (m.classList.contains('acidente') && m.getAttribute('data-local') === localDeInstalacao) {
                            m.classList.add('highlight');
                        }
                    });
                }

                    // Se for um marcador de vão, adiciona um losango de destaque
                    if (this.classList.contains('vao')) {
                        let highlightDiamond = this.querySelector('.highlight-diamond');
                        if (!highlightDiamond) {
                            highlightDiamond = document.createElement('div');
                            highlightDiamond.className = 'highlight-diamond';
                            this.appendChild(highlightDiamond); // Adiciona o losango ao marcador
                        }
                        highlightDiamond.style.display = 'block'; // Mostra o losango
                    } else {
                        // Se não for um vão, remova o destaque diamond
                        let highlightDiamond = this.querySelector('.highlight-diamond');
                        if (highlightDiamond) {
                            highlightDiamond.style.display = 'none'; // Esconde o losango
                        }
                    }

                });
            });
            document.addEventListener('click', function(event) {
                const clickedMarker = event.target.closest('.map-marker');
                if (!clickedMarker) {
            markers.forEach(m => {
                m.classList.remove('highlight');
                const highlightCircle = m.querySelector('.highlight-circle');
                if (highlightCircle) {
                    highlightCircle.style.display = 'none'; // Esconde o círculo
                }
                const highlightDiamond = m.querySelector('.highlight-diamond');
                if (highlightDiamond) {
                    highlightDiamond.style.display = 'none'; // Esconde o losango
                        }
                    });
                }
            });
        });
    </script>
    """
    mapa.get_root().html.add_child(folium.Element(highlight_script))

### Define classe para legenda utilizada no mapa

In [ ]:
# Define a classe Legend (Para que a legenda não desapareça no modo fullscreen)
class Legend(MacroElement):
    def __init__(self, legend_html, marginright, position='bottomright'):
        super(Legend, self).__init__()
        self._name = 'Legend'
        self.legend_html = legend_html
        self.position = position
        self.marginright = marginright
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            L.Control.Legend = L.Control.extend({
                onAdd: function(map) {
                    var div = L.DomUtil.create('div', 'legend-control');  // Assign class here
                    div.innerHTML = `{{this.legend_html}}`;
                    div.style.marginRight = '{{this.marginright}}';
                    return div;
                },
                onRemove: function(map) {
                    // Nothing to do here
                }
            });
            L.control.legend = function(opts) {
                return new L.Control.Legend(opts);
            }
            L.control.legend({ position: '{{this.position}}' }).addTo({{this._parent.get_name()}});
        {% endmacro %}
        """)


## Criação do Mapa

In [ ]:
# calcula onde vai começar centralizado o mapa
media_x = df_mapa["latitude"].mean()
media_y = df_mapa["longitude"].mean()

# Cria o mapa centrado nas coordenadas especificadas
mapa = folium.Map(
    location=[media_x, media_y],
    zoom_start=5,
    control_scale=True,
    dragging=True,
    max_zoom=18,
    min_zoom=4,

)

### Adiciona atributos ao mapa

In [ ]:
# Adiciona o botão de fullscreen
Fullscreen(position='topright', force_separate_button=True).add_to(mapa)

# Adiciona legenda
caminho_legenda_html = ajuste_path('src/core/mapa/assets/legenda.html')
with open(caminho_legenda_html, "r", encoding="utf-8") as arquivo:
    legenda_html = arquivo.read()

legend = Legend(legend_html=legenda_html,
                marginright='5.6vw', position='bottomright')
mapa.add_child(legend)


# Adicionar a camada de polígonos de estados do Brasil
url_poly_brasil = "https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF"

headers = {"Accept": "application/vnd.geo+json"}

mapa_brasil = req.get(url_poly_brasil, headers=headers)

poly_brasil = mapa_brasil.json()
camada_poligonos_brasil = folium.FeatureGroup(name="Polígonos do Brasil")
folium.GeoJson(
    poly_brasil,
    zoom_on_click=True,
    style_function=lambda feature: {
        "fillColor": "#c8d977",
        "color": "#A9A9A9",
        "weight": 2,
        "dashArray": "5, 5",
        "fillOpacity": 0.15,
    },
).add_to(camada_poligonos_brasil)
mapa.add_child(camada_poligonos_brasil)

### Adiciona locais de instalações e acidentes ao mapa

In [ ]:
# Filtra o df_mapa somente com os valores de probabilidade máxima para cada local, ano e mês.
df_mapa_max_prob = df_mapa.loc[df_mapa.groupby(
    ['ano', 'mes', 'local_de_instalacao'])['probabilidade'].idxmax()]

# Adiciona camada ao mapa com a probabilidade máxima para cada local presente no df_mapa para um ano e mês especificado
adiciona_camadas(mapa, filtra_df_por_tempo(df_mapa_max_prob, 6, 2024,),
                 "Exibir probabilidade de 06/2024")


# ADICIONA ACIDENTES

# Filtrar o DataFrame para remover linhas com valores NaN nas colunas de latitude e longitude
df_acidentes = df_mapa.dropna(
    subset=['latitude_acidente', 'longitude_acidente'])

# Adicionar marcadores ao mapa para os acidentes
for _, row in df_acidentes.iterrows():
    if row['potencial_acidente'] <= 8:
        color = '#d7301f'
    elif row['potencial_acidente'] <= 12:
        color = '#fec44f'
    else:
        color = '#006d2c'
    adiciona_acidente(mapa, row, color)


# Adiciona os vãos das Linhas de Tranmissão (LT)
# adiciona_vaos_lt(mapa, df_lt_filtrado, '#525252')

# Adiciona a camada de controle de camadas
# folium.LayerControl().add_to(mapa)

destaca_marcador_selecionado(mapa)

### Salva o html do mapa

In [ ]:
# Gerar o HTML do mapa como uma string
mapa_html = mapa._repr_html_()

# Salvar a string em um arquivo com encoding UTF-8
caminho_assets = ajuste_path('src/core/mapa/assets/mapa.html')

with open(caminho_assets, "w", encoding="utf-8") as arquivo:
    arquivo.write(mapa_html)

# Exibe o mapa
mapa